<a href="https://colab.research.google.com/github/aswinaus/ML/blob/main/Azure_DocumentIntelligence_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install azure-ai-formrecognizer openai

In [7]:
# Step 1: Parse document using Azure Document Intelligence
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
from openai import OpenAI

# Replace with your actual access token
from google.colab import userdata
DOCUMENTINTEL_KEY = userdata.get('DOCUMENTINTEL_KEY')

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

import nest_asyncio
nest_asyncio.apply()


from google.colab import drive
drive.mount('/content/drive')

data_dir = '/content/drive/MyDrive' # Input a data dir path from your mounted Google Drive

# Azure Document Intelligence setup
endpoint = "https://documentsclassifier.cognitiveservices.azure.com/"
key = DOCUMENTINTEL_KEY

document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(DOCUMENTINTEL_KEY)
)

# Analyze a document
with open(f"{data_dir}/RAG/data/10k/lyft_10k_2023.pdf", "rb") as f:
    # This is the name of the pre-trained model being used for the analysis. Azure Document Intelligence provides various pre-built models for different document types (like invoices, receipts, identity documents, etc.).
    # In this case "prebuilt-document" is a general-purpose model that can extract text, layout
    # and other key information from various types of documents.
    poller = document_analysis_client.begin_analyze_document("prebuilt-document", document=f)
    #document=f: This argument provides the document to be analyzed. f is a file handle representing the opened file.
result = poller.result()

# Extract text from document
#extracted_text = "\n".join([page.content for page in result.pages])
extracted_text=result.content

# Step 2: Send extracted content to GPT for classification
client = OpenAI(api_key=OPENAI_API_KEY)

prompt = f"""
Given the following document text, analyze its content.
Identify the underlying financial or tax-related theme, such as compliance, reporting, audit, accounting, policy, corporate finance, personal taxation, investment
or regulatory matters.
If there is a subcategory then make sure subcatgory is included as comma separted values in the response.
On this analysis classify the document into the most appropriate financial or tax-related category
that best represents its primary subject matter.
Also return the number of pages.
include the precide page number where Tax or related  content occurs in the documents.
If the above exists in more than one page have it displayed as comma separated like 1,2
Return JSON with fields: category, confidence, description, Number of Pages, Page Number.
And make sure subcategory is included in the response.
Document:
{extracted_text}
"""

response = client.chat.completions.create(
    model="gpt-4o",  # or "gpt-4.0"
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
```json
{
  "category": "Regulatory Matters",
  "confidence": "High",
  "description": "The document is an SEC Form 10-K, which is an annual report filed by public companies in the United States. The primary subject matter relates to regulatory compliance requirements under the Securities Exchange Act of 1934, including sections about business operations, risk factors, financial statements, and other disclosures.",
  "Number of Pages": 2,
  "Page Number": "1"
}
```
